In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [ ]:
sequence_to_classify = "one day I will see the world"
# sequence_to_classify = "In this work, fibrin (FIB) incorporated injectable alginate dialdehyde (ADA) - gelatin (G) hydrogel was explored as a matrix for liver tissue engineering."
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(sequence_to_classify, candidate_labels)